# Berke Lab Spyglass Process!

## 1: Insert the session and populate relevant tables.

Note this doesn't include ephys + sorting + decoding

In [3]:
import spyglass.common as sgc
import spyglass.position as sgp
import spyglass.data_import as sgi
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from hex_maze_behavior import populate_all_hexmaze, populate_hex_position
from fiber_photometry import populate_all_fiber_photometry

nwb_file_name = "IM-1478_20220719.nwb"

# When we run insert_session, spyglass creates a copy of the nwbfile named file name + _
# We use nwb_copy_file_name to make it explicit which one to use.
# Everything besides sgi.insert_session should use the copy file name
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)

## Step 1. 
# Insert session into spyglass (populates all common tables)
sgi.insert_sessions(nwb_file_name)

## Step 2.
# Once the session has been inserted, insert into basic hex maze tables
# (populates HexMazeBlock, HexMazeBlock.Trial, HexMazeConfig, HexCentroids tables)
populate_all_hexmaze(nwb_copy_file_name)

## Step 3. 
# Process position using default parameters
# We only have one epoch (epoch 0) for Berke lab, so "pos 0 valid times" is always the interval list name
interval_list_name = "pos 0 valid times"

# Our position selection key includes the nwbfile name, the interval to process, and the parameters to use
position_selection_key = {
    "nwb_file_name": nwb_copy_file_name,
    "interval_list_name": interval_list_name,
    "trodes_pos_params_name": "default", # for now we always use default parameters
}
# To associate a set of parameters with a given interval, insert them into the `TrodesPosSelection` table
# Note "trodes position" really just means any imported unprocessed position
sgp.v1.TrodesPosSelection.insert1(position_selection_key, skip_duplicates=True)

# We can run the pipeline for our chosen interval/parameters by using the `TrodesPosV1.populate`
# Each NWB file, interval, and parameter set is now associated with a new analysis file and object ID.
sgp.v1.TrodesPosV1.populate(position_selection_key)

## Step 4. 
# Insert into HexPositionSelection and HexPosition tables
# (must have run populate_all_hexmaze (Step 2) and have an entry in PositionOutput table (Step 3))
populate_hex_position(nwb_copy_file_name)

## Step 5.
# Insert into all photometry-related tables
populate_all_fiber_photometry(nwb_copy_file_name)


/home/scrater/dev/spyglass/src/spyglass/data_import/insert_sessions.py:66: UserWarning: Cannot insert data from IM-1478_20220719.nwb: IM-1478_20220719_.nwb is already in Nwbfile table.
  warnings.warn(


Populating HexPosition for 1 entries in IM-1478_20220719_.nwb


[13:45:56][INFO] Spyglass: Populating photometry device tables from IM-1478_20220719_.nwb
INFO:spyglass:Populating photometry device tables from IM-1478_20220719_.nwb
[13:45:57][INFO] Spyglass: Inserted excitation sources ['Thorlabs Blue LED', 'Thorlabs Purple LED']
INFO:spyglass:Inserted excitation sources ['Thorlabs Blue LED', 'Thorlabs Purple LED']
[13:45:57][INFO] Spyglass: Inserted photodetectors ['Doric iFMC7-G2 (7 ports Fluorescence Mini Cube - Three Fluorophores)']
INFO:spyglass:Inserted photodetectors ['Doric iFMC7-G2 (7 ports Fluorescence Mini Cube - Three Fluorophores)']
[13:45:57][INFO] Spyglass: Inserted optical fibers ['Doric 0.66mm Flat 40mm Optic Fiber']
INFO:spyglass:Inserted optical fibers ['Doric 0.66mm Flat 40mm Optic Fiber']
[13:45:57][INFO] Spyglass: Populating fiber photometry series from IM-1478_20220719_.nwb
INFO:spyglass:Populating fiber photometry series from IM-1478_20220719_.nwb
[13:45:57][INFO] Spyglass: Inserted photometry series ['raw_green', 'raw_refere

## 2. Look at all the tables we populated!

wow, so many!

In [4]:
from hex_maze_behavior import HexMazeBlock, HexCentroids, HexMazeConfig, HexPositionSelection, HexPosition
from fiber_photometry import ExcitationSource, OpticalFiber, Photodetector, FiberPhotometrySeries
from spyglass.common import IntervalList

key = {"nwb_file_name" : nwb_copy_file_name}
#key = 'nwb_file_name LIKE "IM-%"'

print("------------------------- Hex maze tables! ------------------------- \n")

# Show the HexMazeBlock table
print("HexMazeBlock table:")
display(HexMazeBlock() & key)

# Show the Trial part table
print("HexMazeBlock().Trial part table:")
display(HexMazeBlock().Trial() & key)

# Show the HexCentroids table (nwb_file_name is key)
print("HexCentroids table:")
display(HexCentroids() & key)

# Show the HexCentroids part table (nwb_file_name and hex)
print("HexCentroidsPart:")
display((HexCentroids().HexCentroidsPart()) & key)

# All config_ids for blocks in your session will also now exist in the HexMazeConfig table
# Show the HexMazeConfig table
print("HexMazeConfig table (not restricted by nwb_file_name, includes all sessions):")
display(HexMazeConfig())

# Show the HexPositionSelection table
print("HexPositionSelection table:")
display(HexPositionSelection() & key)

# Show the HexPosition table
print("HexPosition table:")
display((HexPosition()) & key)

print("------------------------- Photometry tables! -------------------------\n")

print("FiberPhotometrySeries table")
display(FiberPhotometrySeries() & key)

print("OpticalFiber table (not restricted by nwb_file_name, includes all sessions):")
display(OpticalFiber() & key)

print("Photodetector table (not restricted by nwb_file_name, includes all sessions):")
display(Photodetector() & key)

print("ExcitationSource table (not restricted by nwb_file_name, includes all sessions):")
display(ExcitationSource() & key)

print("------------------------- Interval Lists for this nwb! ------------------------- \n")

print("IntervalList table:")
display(IntervalList() & key)


------------------------- Hex maze tables! ------------------------- 

HexMazeBlock table:


nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch,config_id maze configuration as a string,interval_list_name descriptive name of this interval list,p_a probability of reward at port A,p_b probability of reward at port B,p_c probability of reward at port C,num_trials number of trials in this block,task_type 'barrier shift' or 'probabilty shift'
IM-1478_20220719_.nwb,0,1,"8,9,16,18,23,27,35,42,44",epoch0_block1,90.0,10.0,50.0,65,barrier change
IM-1478_20220719_.nwb,0,2,"8,9,18,21,23,27,35,42,44",epoch0_block2,90.0,10.0,50.0,62,barrier change
IM-1478_20220719_.nwb,0,3,"8,9,18,21,23,24,27,42,44",epoch0_block3,90.0,10.0,50.0,58,barrier change


HexMazeBlock().Trial part table:


nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch,block_trial_num trial number within the block,interval_list_name descriptive name of this interval list,epoch_trial_num trial number within the epoch,reward if the rat got a reward,"start_port A, B, or C","end_port A, B, or C","opto_cond description of opto condition, if any (delay / no_delay)","poke_interval np.array of [poke_in, poke_out]",duration trial duration in seconds
IM-1478_20220719_.nwb,0,1,1,epoch0_block1_trial1,1,0,None,C,None,=BLOB=,10.5167
IM-1478_20220719_.nwb,0,1,2,epoch0_block1_trial2,2,1,C,B,None,=BLOB=,31.1152
IM-1478_20220719_.nwb,0,1,3,epoch0_block1_trial3,3,1,B,C,None,=BLOB=,21.6368
IM-1478_20220719_.nwb,0,1,4,epoch0_block1_trial4,4,1,C,A,None,=BLOB=,54.662
IM-1478_20220719_.nwb,0,1,5,epoch0_block1_trial5,5,0,A,B,None,=BLOB=,9.01763
IM-1478_20220719_.nwb,0,1,6,epoch0_block1_trial6,6,0,B,C,None,=BLOB=,17.0026
IM-1478_20220719_.nwb,0,1,7,epoch0_block1_trial7,7,1,C,A,None,=BLOB=,81.1043
IM-1478_20220719_.nwb,0,1,8,epoch0_block1_trial8,8,0,A,B,None,=BLOB=,22.7089
IM-1478_20220719_.nwb,0,1,9,epoch0_block1_trial9,9,1,B,C,None,=BLOB=,32.461
IM-1478_20220719_.nwb,0,1,10,epoch0_block1_trial10,10,1,C,A,None,=BLOB=,18.5386


HexCentroids table:


...

photodetector_name,manufacturer,model,description,detector_type,detected_wavelength_in_nm
Doric iFMC7-G2 (7 ports Fluorescence Mini Cube - Three Fluorophores),Doric,iFMC7-G2,https://neuro.doriclenses.com/products/fmc7?productoption%5BPort%20Configuration%5D=Built-in%20DETECTOR,Silicon photodiode,960.0


ExcitationSource table (not restricted by nwb_file_name, includes all sessions):


excitation_source_name,manufacturer,model,illumination_type,excitation_wavelength_in_nm
Thorlabs Blue LED,Thorlabs,M470F3,LED,470.0
Thorlabs Purple LED,Thorlabs,M405FP1,LED,405.0


------------------------- Interval Lists for this nwb! ------------------------- 

IntervalList table:


nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,valid_times numpy array with start/end times for each interval,"pipeline type of interval list (e.g. 'position', 'spikesorting_recording_v1')"
IM-1478_20220719_.nwb,00_r1,=BLOB=,
IM-1478_20220719_.nwb,04d1f615-9bf7-4e95-a72f-07de196c4c62,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,0d12bcb9-1310-4ebb-a730-a9e398734fb7,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,190a8da9-a15f-49b3-8c09-c760803a1358,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,1dd28801-d371-4213-ac01-e07f22f796af,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,281d02aa-28c1-4a76-be50-da87dccbab27,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,337e852e-b9b0-4f7b-a6d6-07d48cc14e48,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,4237465f-bee3-414d-b61a-5afc33dfdc8d,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,4b6d2881-325a-416d-8dab-2eb4a1d8c12d,=BLOB=,spikesorting_recording_v1
IM-1478_20220719_.nwb,641515b7-a1c2-4883-8053-6383a9d80fec,=BLOB=,spikesorting_recording_v1


## 3. Fetch position data (x,y position and hex position)

In [12]:
from fiber_photometry import FiberPhotometrySeries
from hex_maze_behavior import HexPositionSelection, HexPosition
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from spyglass.position import PositionOutput
from spyglass.common import Nwbfile, IntervalList
import spyglass.common as sgc
from spyglass.position import PositionOutput
from hex_maze_behavior import HexPosition, HexMazeBlock

nwb_file_name = "IM-1478_20220719_.nwb"
epoch = 0

# Get all blocks for this epoch so we can filter to only valid times
blocks = (HexMazeBlock & {"nwb_file_name": nwb_file_name, "epoch": epoch}).fetch()
first_block = blocks[0]
last_block = blocks[-1]

first_block_start, first_block_end = (IntervalList & {
    'nwb_file_name': nwb_file_name,
    'interval_list_name': first_block['interval_list_name']
}).fetch1('valid_times')[0]

last_block_start, last_block_end = (IntervalList & {
    'nwb_file_name': nwb_file_name,
    'interval_list_name': last_block['interval_list_name']
}).fetch1('valid_times')[0]

# Get position merge ID for this epoch
# There may be multiple if position was processed multiple times so we just fetch1 and hope for the best
interval_list_name = f"pos {epoch} valid times"
position_output_key = {
    "nwb_file_name": nwb_file_name,
    "interval_list_name": interval_list_name
}
merge_id = (PositionOutput.merge_get_part(position_output_key)).fetch1("KEY")

# Get processed xy position from the PositionOutput table
xy_position_df = (PositionOutput & {"merge_id": merge_id["merge_id"]}).fetch1_dataframe()

# Get hex position from the HexPosition table
# Note in the HexPosition table, we have to use 'pos_merge_id' instead of 'merge_id'
hex_position_df = (HexPosition & {"pos_merge_id": merge_id["merge_id"]}).fetch1_dataframe()

print("x,y position from PositionOutput table")
display(xy_position_df)

print("Hex position from HexPosition table")
display(hex_position_df)
# NOTE hex position will probably show -1 and None values because the start/end of the dataframe are
# outside of block time boundaries so position was not assigned to a hex here. The filtered dataframe
# below should show real values as expected

# Combine x,y position with assigned hex position
full_position_df = xy_position_df.join(hex_position_df, on='time')

# Filter position data to only include times between first block start and last block end
mask = (full_position_df.index >= first_block_start) & (full_position_df.index <= last_block_end)
full_position_df  = full_position_df .loc[mask]

print("Combined position (filtered to only valid times)")
display(full_position_df )


x,y position from PositionOutput table


,video_frame_ind,position_x,position_y,orientation,velocity_x,velocity_y,speed
time,,,,,,,
0.712208,0,26.037442,25.670653,1.567180,-3.185546,0.203464,3.192037
0.733519,1,25.848425,25.620620,1.430961,-3.074016,1.870411,3.598335
0.782489,2,26.007758,25.816855,1.289803,-2.584660,4.637361,5.309009
0.847202,3,26.079524,25.846221,1.421020,-2.778857,7.359527,7.866682
0.910277,4,25.923258,27.283797,1.455922,-3.783289,8.490328,9.295103
...,...,...,...,...,...,...,...
7141.946028,107121,67.221305,43.334339,0.683602,0.100016,0.183027,0.208571
7142.010011,107122,67.198546,43.317982,0.691598,-0.399310,-0.489317,0.631569
7142.073610,107123,67.175050,43.165421,0.678765,-0.801968,-0.849177,1.168013


Hex position from HexPosition table


,hex,hex_including_sides,distance_from_centroid
time,,,
0.712208,-1,None,-1.0
0.733519,-1,None,-1.0
0.782489,-1,None,-1.0
0.847202,-1,None,-1.0
0.910277,-1,None,-1.0
...,...,...,...
7141.946028,-1,None,-1.0
7142.010011,-1,None,-1.0
7142.073610,-1,None,-1.0


Combined position (filtered to only valid times)


,video_frame_ind,position_x,position_y,orientation,velocity_x,velocity_y,speed,hex,hex_including_sides,distance_from_centroid
time,,,,,,,,,,
27.245063,399,129.037837,104.683188,-2.810178,14.958602,31.905975,35.238487,29,29,4.867746
27.309610,400,129.779214,106.958650,-3.028090,17.907319,29.137373,34.200271,29,29,6.662438
27.372864,401,131.419868,108.309498,-2.877040,20.259540,25.854427,32.846619,34,34,5.215510
27.436989,402,132.631779,109.882279,-2.827626,21.534849,23.602111,31.950107,34,34,3.485957
27.516921,403,134.531964,111.129663,-2.569731,21.039270,22.195119,30.582253,34,34,1.485978
...,...,...,...,...,...,...,...,...,...,...
7070.491045,106049,101.410840,15.117798,0.897172,-0.032487,-0.217291,0.219706,1,1,3.632302
7070.554772,106050,101.446830,15.115658,0.906959,1.528549,-0.856999,1.752400,1,1,3.610123
7070.619010,106051,101.862811,14.907614,1.109121,2.565169,-1.365858,2.906142,1,1,3.522002


## Extra notes etc

My notes on the process for inserting into PositionOutput

In [ ]:
import spyglass.common as sgc
import spyglass.position as sgp
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename

nwb_file_name = "IM-1478_20220719.nwb"

# When we run insert_session, spyglass creates a copy of the nwbfile named file name + _
# We use nwb_copy_file_name to make it explicit which one to use
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)

# Look at the session in the raw position table
display(sgc.common_behav.RawPosition() & {"nwb_file_name": nwb_copy_file_name})

# Look at our interval lists
display(sgc.IntervalList & {"nwb_file_name": nwb_copy_file_name})

# We only have one epoch for Berke Lab (epoch 0), so this is always the interval list name
interval_list_name = f"pos 0 valid times"
raw_position_df = (sgc.RawPosition() & {"nwb_file_name": nwb_copy_file_name, "interval_list_name": interval_list_name}).fetch1_dataframe()
display(raw_position_df)

# Look at default names of valid position processing params (we will use "default")
display(sgp.v1.TrodesPosParams())

# To associate a set of parameters with a given interval, insert them into the `TrodesPosSelection` table.
trodes_s_key = {
    "nwb_file_name": nwb_copy_file_name,
    "interval_list_name": interval_list_name,
    "trodes_pos_params_name": "default",
}
sgp.v1.TrodesPosSelection.insert1(
    trodes_s_key,
    skip_duplicates=True,
)

# We can run the pipeline for our chosen interval/parameters by using the `TrodesPosV1.populate`
# Each NWB file, interval, and parameter set is now associated with a new analysis file and object ID.
sgp.v1.TrodesPosV1.populate(trodes_s_key)

[2025-05-16 15:22:34,412][INFO]: DataJoint 0.14.4 connected to scrater@lmf-db.cin.ucsf.edu:3306


nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list
IM-1478_20220719_.nwb,pos 0 valid times


nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,valid_times numpy array with start/end times for each interval,"pipeline type of interval list (e.g. 'position', 'spikesorting_recording_v1')"
IM-1478_20220719_.nwb,00_r1,=BLOB=,
IM-1478_20220719_.nwb,pos 0 valid times,=BLOB=,position
IM-1478_20220719_.nwb,raw data valid times,=BLOB=,


,xloc1,yloc1,xloc2,yloc2
time,,,,
0.712208,81.785416,88.307152,81.729721,72.904549
0.733519,82.316483,88.636253,80.011627,72.261238
0.782489,83.965561,89.037750,79.363159,73.092102
0.847202,83.222534,89.989288,80.556877,72.324982
0.910277,82.444191,94.729355,80.353867,76.612892
...,...,...,...,...
7141.946028,218.717239,142.295547,203.432556,129.844101
7142.010011,218.696426,142.388458,203.310440,129.648468
7142.073610,218.689545,141.798706,203.169769,129.280136


trodes_pos_params_name name for this set of parameters,params
decoding_xs,=BLOB=
default,=BLOB=
default_decoding,=BLOB=
default_decoding_4ms,=BLOB=
default_led0,=BLOB=
default_lowerspeed,=BLOB=
default_speed35,=BLOB=
dlc_head_body,=BLOB=
double_led,=BLOB=
double_led_CD,=BLOB=


[15:23:00][INFO] Spyglass: Computing position for: {'nwb_file_name': 'IM-1478_20220719_.nwb', 'interval_list_name': 'pos 0 valid times', 'trodes_pos_params_name': 'default'}
[15:23:22][INFO] Spyglass: Writing new NWB file IM-1478_20220719_R2UO2ONXEB.nwb
[15:23:24][INFO] Spyglass: No video frame index found. Assuming all camera frames are present.


{'success_count': 1, 'error_list': []}